# RAG-SQL

RAG (Retrieval-Augmented Generation) with SQL is a method used to combine natural language processing (NLP) models, like GPT, with structured query retrieval from databases using SQL (Structured Query Language). The goal is to leverage both the power of language models to understand and generate human-like text and the accuracy of SQL queries to retrieve data from structured sources, like relational databases.

![DESCRIPTION](ragsql.webp)

### Input the Question or Query:
The process begins with the user asking a question or making a request in natural language. For example, the question might be:

"What is the total number of movies available in the table?"

### Preprocess the Input

In some implementations, the input may need preprocessing. This could involve Breaking down the question into smaller units (words or tokens)  Recognizing the key parts of the query, such as the “total number,” “movies available,” and “in the table.” Identifying what kind of SQL query is needed based on the user’s request.
### Retrieve Schema Information from the Database
Before generating the SQL query, the system needs to understand the structure of the database to know which tables and columns contain the relevant information.The language model (or another retrieval system) retrieves the schema of the database, including table names, column names, and relationships between them.
For example, the model might need to know that the companies table has a sector column and a revenue column, and there’s a year column somewhere indicating the timeframe.
###  Translate Natural Language into SQL Query
The key step is converting the natural language question into an SQL query. A language model (e.g., GPT or similar) is used to Understand the user's intent. Generate the SQL query based on the natural language input and the schema retrieved from the database.
For the earlier question, the model might generate an SQL query like this

SELECT COUNT(`ID`)
AS TotalMovies FROM telugumovies_dataset;

### Execute the SQL Query
Once the SQL query is generated, it is executed against the database. The system sends the SQL command to the relational database where the data is stored.

The database processes the query and returns the relevant results, such as the average revenue of companies in the tech sector over the last five years.

### Post-process the Results
The results returned by the SQL query may require some post-processing to make them more human-readable.For example Formatting the data (e.g., rounding numbers or displaying results in a table) Adding contextual information (e.g., explaining what the result represents).For instance, if the query returns the total number of movies in the table, the system might phrase it as:

“There are 1400 movies in the dataset.”
### Generate the Final Response
After retrieving the structured data, the final response is generated. The system uses the information retrieved from the SQL query to craft a complete, coherent response that answers the original question.


### Return the Response to the User
The final step is delivering the response to the user, either in a conversational format (text) or a more structured format (table, graph, etc.).
This fusion of NLP models and SQL allows for powerful, conversational-style data retrieval from structured databases.

### Installing dependencies

In [1]:
!pip install langchain langchain-experimental openai pymysql

#### Langchain
langchain is a framework designed for building applications that combine large language models (LLMs) with various external data sources. It is particularly useful for creating chains of interactions between language models and other systems such as APIs, databases, and document retrieval systems.
#### langchain-experimental
This package contains experimental features and components that are not part of the stable langchain release. These features might still be under development or testing but provide access to cutting-edge methods that can be useful for advanced use cases.
#### openai
The openai Python package is used to interact with OpenAI’s API, enabling you to use their language models (like GPT-3, GPT-4) for various tasks such as text generation, question-answering, summarization, and more.

### Import necessary packages

In [2]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

"SQLDatabase" is a utility provided by langchain that allows you to connect and interact with SQL databases. It abstracts the process of connecting to the database, running SQL queries, and retrieving results. You can use this class to create a connection to a SQL database (e.g., MySQL, PostgreSQL, SQLite) and use it within langchain to retrieve data for natural language queries.

The "OpenAI" class provides an interface to interact with OpenAI’s language models (e.g., GPT-3, GPT-4). It allows you to send prompts to OpenAI’s API and receive natural language responses.This is how you access OpenAI's LLMs to generate responses, complete tasks, or convert natural language input into structured formats like SQL.

"SQLDatabaseChain" is part of the langchain-experimental package, which is focused on experimental features. This class allows you to create a chain that specifically interacts with SQL databases. It uses language models to generate SQL queries from natural language and retrieve relevant data from the database.It connects the language model with the SQL database and enables it to process natural language queries by converting them into SQL queries, executing them, and returning results in a conversational format.

"PromptTemplate" is used to structure the prompts that will be sent to the language model. It allows you to define templates with placeholders and then fill in those placeholders dynamically when generating the prompt.You can create customizable prompts for tasks like converting a natural language request into a SQL query, summarizing text, or answering questions.

"HumanMessagePromptTemplate" This class helps structure a prompt specifically for chat-based conversations. It supports the creation of prompts that resemble human messages and are used in chat-based interactions with LLMs.  You use this template to generate human-like input, which will be processed by the LLM in a conversational format.

"ChatOpenAI" is specifically tailored for chat-based interactions using OpenAI’s models (like GPT). It facilitates a back-and-forth conversation with the model. It allows you to send multiple conversational turns (messages) to the OpenAI API and get responses in a structured conversation format.

"HumanMessage" and "SystemMessage" are part of langchain's message structure, representing the different types of messages in a conversation. HumanMessage Represents a message sent by the user (i.e., human input).SystemMessage Represents a message from the system that guides the conversation (often used to set context for the LLM). These classes are useful for building conversational agents where multiple types of messages (from human and system) need to be processed in context by the LLM.

These imports work together to create an application where, You can ask natural language questions, and the system will convert them into SQL queries using OpenAI’s language models. The converted SQL queries are executed on a connected SQL database (via SQLDatabaseChain), and the results are retrieved. The interaction can be conversational, where human and system messages guide the behavior and context of the language model using ChatOpenAI.

### Initialize the LLM

In [3]:
OPENAI_API_KEY = ""
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

C:\Users\Sreedhar\AppData\Local\Temp\ipykernel_20972\1703113971.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)


The API key is provided when you sign up for OpenAI's services and is required to access their language models (like GPT-3 or GPT-4). This key will be used later to interact with OpenAI’s language models through the ChatOpenAI class."temperature=0" This controls the "creativity" or randomness of the language model. A value of 0 makes the model more deterministic (it will produce the same response for the same input)."openai_api_key=OPENAI_API_KEY" This passes the OpenAI API key to authenticate your requests when communicating with the language model.

### DataBase Setup

In [5]:
host = 'localhost'
port = '3307'
username = 'root'
#password = ''
database_schema = 'mydb'
mysql_uri = f"mysql+pymysql://{username}@{host}:{port}/{database_schema}"
#mysql_uri = f"mysql+pymysql://{username:password}@{host}:{port}/{database_schema}"
db = SQLDatabase.from_uri(mysql_uri, include_tables=['telugumovies_dataset'],sample_rows_in_table_info=2)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

host = 'localhost': Specifies the server where the MySQL database is hosted. In this case, it's running on the local machine (localhost).).
port = '3306': This is the default port number for MySQL.
username = 'root': The username used to access the database. In this case, it's the root user.
password = 'password': The password for the root user to access the database.
database_schema = 'database_name': The name of the database schema you're connecting to.

This creates an instance of SQLDatabase, which establishes a connection to the MySQL database using the previously constructed mysql_uri. "mysql_uri" is the URI containing the credentials and details to connect to the MySQL database."include_tables=['table_name']" This specifies that only the table job_details should be included for interaction in the context of this language model and database connection. The model will focus on generating SQL queries and interacting with this specific table."sample_rows_in_table_info=2" This option will fetch and display 2 sample rows of data from the job_details table to help the language model better understand the structure and content of the table. This is particularly useful when generating SQL queries.

"db_chain" creates a SQLDatabaseChain, which is responsible for converting natural language queries into SQL queries, running them against the MySQL database, and retrieving results. The chain uses both the language model (llm) and the database connection (db)."llm" The language model (ChatOpenAI) that will interpret natural language inputs and generate the corresponding SQL queries. "db" The SQLDatabase instance that contains the MySQL connection and table information. "verbose=True" This enables detailed logging of the interactions, which will print out the generated SQL queries, execution steps, and results. It’s useful for debugging and understanding how the language model interacts with the database.The SQLDatabaseChain takes that generated SQL query and runs it against the MySQL database connected via the SQLDatabase instance. 



In [6]:
def retrieve_from_db(query: str) -> str:
    db_context = db_chain(query)
    db_context = db_context['result'].strip()
    return db_context

#### Function Definition
This function is defined to take a natural language query as input (in the form of a string) and return the result from the database."query: str" This parameter accepts a natural language query in string format. For example, a query might be, "What is the total number of movies available in the table?". The function will return a string, which will contain the database result after processing.

#### Sending Query to the Database Chain
The db_chain is an instance of SQLDatabaseChain created earlier. It processes the natural language input (query) using an AI language model and converts it into an appropriate SQL query.
The SQL query is then executed against the connected database.
The result of that SQL query is returned as a dictionary (which includes additional metadata).

#### Extracting and Cleaning the Result
The result of the SQL query is stored in the db_context dictionary under the key 'result'. This contains the outcome of the query (e.g., the number of jobs or some specific data).
.strip() method removes any unnecessary whitespace (like spaces or newlines) from the start and end of the string to ensure the result is clean and ready to be returned.
#### Returning the Result
After the query is processed, converted into SQL, executed, and the result is cleaned, it is sent back as the output of the function.

In [7]:
def generate(query: str) -> str:
    db_context = retrieve_from_db(query)
    
    system_message = """You are a movie database expert specializing in Telugu cinema.
        Your task is to answer users' questions by providing relevant information from a database of Telugu movies.
        The database contains the following information for each movie:
        - Name of the movie
        - Year of release
        - Certificate given by the censor board
        - Movie genre
        - A brief description or plot of the movie
        - Duration in minutes
        - IMDb Rating
        - Number of people who rated the movie
        
        Example:
        
        Input:
        Which Telugu movies released in 2021 have the highest IMDb ratings?
        
        Context:
        The Telugu movies released in 2021 with the highest IMDb ratings are:
        1. Movie A - IMDb Rating: 8.5
        2. Movie B - IMDb Rating: 8.3
        
        Output:
        The highest-rated Telugu movies released in 2021 are Movie A (IMDb Rating: 8.5) and Movie B (IMDb Rating: 8.3).
        """
    
    human_qry_template = HumanMessagePromptTemplate.from_template(
        """Input:
        {human_input}
        
        Context:
        {db_context}
        
        Output:
        """
    )
    messages = [
      SystemMessage(content=system_message),
      human_qry_template.format(human_input=query, db_context=db_context)
    ]
    response = llm(messages).content
    return response

This generate function is designed to take a natural language query (related to Telugu movies) as input, interact with a database to retrieve relevant information, and then use an AI language model to generate a human-readable response.

The generate function processes a user's natural language query, fetches relevant data from the database, and then uses an AI language model to produce a well-structured, natural language response based on that data.
#### Retrieve Data from Database
db_context = retrieve_from_db(query) This calls the previously defined retrieve_from_db function, which takes the user's query, generates a SQL query, and retrieves relevant information from the database. The resulting data is stored in db_context.The query is processed by retrieve_from_db, and the corresponding result from the database is fetched and assigned to db_context.
#### Define System Message
This string defines the system message that provides instructions to the AI language model (LLM). It establishes the AI’s role and context for generating responses.
The AI is instructed to act as a "movie database expert specializing in Telugu cinema."
The message explains the kind of data the database contains, such as movie names, release years, genres, IMDb ratings, etc.
It also provides an example of how the AI should handle input (from the user) and the corresponding output.
This helps guide the model to structure its response in a specific way (focusing on relevant information from the query).
#### Create a Human Message Template
This defines the template for the human message that will be passed to the language model (LLM) to format the user input and database result.{human_input}: This is where the user's natural language query (like "Which Telugu movies released in 2021 have the highest IMDb ratings?") will be inserted.
{db_context}: This is where the database result (db_context) from the earlier SQL query will be inserted (e.g., "1. Movie A - IMDb Rating: 8.5").The template format structures the human message in three parts, "Input" The original question from the user."Context" The result retrieved from the database, which provides relevant data."Output" The expected answer, which the language model will generate.
#### Create the Messages
This creates the list of messages that will be passed to the AI language model (llm) for generating a response."SystemMessage(content=system_message)" This is the first message, which provides the AI with the context (i.e., that it is acting as a movie database expert, as defined earlier).
"human_qry_template.format(human_input=query, db_context=db_context)" This creates the human message by formatting the user's query and the retrieved database context into the template. The AI will use this to understand the user’s input and the relevant data from the database.
Together, these two messages provide the necessary information for the AI to generate a coherent response.
#### Generate and return the Response from LLM
The llm(messages) call sends both the system and human messages to the OpenAI model.
The model processes the query, the database context, and the system instructions to produce a natural language response that answers the user’s query.
.content retrieves the actual response content generated by the model.After the AI processes the input and the database context, it generates a final output, which is returned as the response to the user.

In [8]:
generate("How many movies are there?")

C:\Users\Sreedhar\AppData\Local\Temp\ipykernel_20972\3606437300.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  db_context = db_chain(query)




> Entering new SQLDatabaseChain chain...
How many movies are there?
SQLQuery:SELECT COUNT(`ID`) AS TotalMovies FROM telugumovies_dataset;
SQLResult: [(1400,)]
Answer:There are 1400 movies in the dataset.
> Finished chain.


C:\Users\Sreedhar\AppData\Local\Temp\ipykernel_20972\3798320535.py:44: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages).content


'There are 1400 movies in the dataset.'

In [9]:
generate("How many movies are there with Action Genre?")



> Entering new SQLDatabaseChain chain...
How many movies are there with Action Genre?
SQLQuery:SELECT COUNT(`ID`) AS `Total_Action_Movies` FROM telugumovies_dataset WHERE `Genre` LIKE '%Action%'
SQLResult: [(630,)]
Answer:There are 630 movies with Action Genre.
> Finished chain.


'There are 630 movies with Action Genre.'

In [10]:
generate("How many movies in year 2015?")



> Entering new SQLDatabaseChain chain...
How many movies in year 2015?
SQLQuery:SELECT COUNT(`ID`) AS `TotalMovies`
FROM telugumovies_dataset
WHERE `Year` = '2015';
SQLResult: [(61,)]
Answer:There are 61 movies in the year 2015.
> Finished chain.


'There are 61 movies in the year 2015.'

# IT will take the column names and values from the question
# it will take use of OPEN AI models to prepare the query
# it will ask the database with the prepared query
# the response will also be converted back to the Description